<a href="https://colab.research.google.com/github/Dutta-SD/Machine_Learning/blob/master/Trell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trell IPython Notebook for prediction

In [1]:
!pip install tpot

     |████████████████████████████████| 92kB 6.4MB/s 
     |████████████████████████████████| 163kB 10.8MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=20801711b25fe200831cfc3baccc0eb5e48b1cad4e11cfdfa1f69184e34e4d6e
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [22]:
!pip install cuML

  ERROR: Failed building wheel for cuML
  Running setup.py clean for cuML
Failed to build cuML
    Running setup.py install for cuML ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-0zzshzo7/cuML/setup.py'"'"'; __file__='"'"'/tmp/pip-install-0zzshzo7/cuML/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-fo8u3gi8/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [10]:
import pandas as pd
import numpy as np

In [11]:
train = pd.read_csv('https://dphi.s3.ap-south-1.amazonaws.com/dataset/train_age_dataset.csv' )
test = pd.read_csv('https://dphi.s3.ap-south-1.amazonaws.com/dataset/test_age_dataset.csv')

## Data Exploration and Preprocessing

In [12]:
'''features = np.array([ 4.42929101, 38.3070717 ,  0.30203578,  0.66176128,  2.61907456,
        1.39784688,  1.60987848,  1.24007281,  0.06248623,  0.23250896,
        0.87295433,  3.14455138,  2.71942532,  3.10195679,  2.62579568,
        0.30790092, 17.73147312,  2.80814877,  0.78053732,  2.06225222,
        2.88141884,  1.44015008,  1.78029997,  1.8217638 ,  2.10248143,
        2.95686235])
features_to_select = [True if i > 2.0 else False for i in features]''';

In [13]:
# test = test.loc[:, features_to_select]

X, y = train.iloc[:, :-1], train.iloc[:, -1]
## Preprocess for NN
# X = X.loc[:, features_to_select]
# y = y

## Change y

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  test_size = 0.25,
                                                  random_state = 10,
                                                  shuffle = True)

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [ ]:
from tpot import TPOTClassifier
model = TPOTClassifier(
    generations=5,
   population_size=20,
   cv=5,
   verbosity=2,
   max_time_mins=10)
model.fit(X_train, y_train)



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.


In [19]:
model.score(X_val, y_val)

0.633128784159712

In [ ]:

from sklearn.decomposition import PCA
from sklearn.metrics import f1_score

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    verbose = True,
     task_type = 'GPU',
      class_weights = [0.5, 8, 8, 8]
      )
preds = model.fit(X_train, y_train).predict(X_val)
f1_score(y_val, preds, average = 'micro')

Learning rate set to 0.213442
0:	learn: 1.2084208	total: 19.7ms	remaining: 19.7s
1:	learn: 1.1128303	total: 37.5ms	remaining: 18.7s
2:	learn: 1.0447921	total: 54.4ms	remaining: 18.1s
3:	learn: 0.9970598	total: 72ms	remaining: 17.9s
4:	learn: 0.9584696	total: 89.2ms	remaining: 17.7s
5:	learn: 0.9302684	total: 107ms	remaining: 17.7s
6:	learn: 0.9107041	total: 129ms	remaining: 18.2s
7:	learn: 0.8920117	total: 149ms	remaining: 18.5s
8:	learn: 0.8817755	total: 167ms	remaining: 18.4s
9:	learn: 0.8700410	total: 184ms	remaining: 18.2s
10:	learn: 0.8585973	total: 199ms	remaining: 17.9s
11:	learn: 0.8511891	total: 214ms	remaining: 17.6s
12:	learn: 0.8448640	total: 230ms	remaining: 17.5s
13:	learn: 0.8422894	total: 245ms	remaining: 17.3s
14:	learn: 0.8389406	total: 261ms	remaining: 17.2s
15:	learn: 0.8320568	total: 276ms	remaining: 17s
16:	learn: 0.8276767	total: 291ms	remaining: 16.8s
17:	learn: 0.8260961	total: 308ms	remaining: 16.8s
18:	learn: 0.8216100	total: 322ms	remaining: 16.6s
19:	learn:

0.7286996127202313

In [ ]:
SUB_NUM = 15

### unnamed:0, this is index of the continuous dataset from which data is taken.
### Can be a possible data leak
# test = test.iloc[:, 1:]

predictions = pd.DataFrame()

X_test_final = sc.transform(test)

finalModel = model

finalPreds = finalModel.predict(X_test_final)

predictions['prediction'] = finalPreds.flatten()

predictions.to_csv(f'predictions_{SUB_NUM}.csv', index = False)